In [4]:
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [1]:
pip install transformers


     |████████████████████████████████| 2.5MB 30.0MB/s 
     |████████████████████████████████| 3.3MB 32.8MB/s 
     |████████████████████████████████| 901kB 33.1MB/s 


In [5]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class TerraDataset(Dataset):


    def __init__(self, file):
        self.file = file
        self.strings = self.load()[0]
        self.predict = torch.tensor(self.load()[1])
        self.tokens = self.transform()

    def load(self):
      data = [json.loads(d) for d in open(self.file, encoding="utf-8")]
      s1 = []
      s2 = []
      for a in data:
        s1.append( a['premise']+' ~> '+a['hypothesis'])
        if a['label']== 'entailment':
          s2.append(1)
        else:
          s2.append(0)
      return [s1,s2]

    def transform(self):
      model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
      tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
      input_ids = []
      for i,string in enumerate(self.strings):
        input_ids.append(tokenizer(string, return_length=True))
      return input_ids


    def __len__(self):
        return len(self.strings)


    def __getitem__(self, idx):
        return self.tokens[idx]

In [6]:
from transformers import DataCollatorForLanguageModeling
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm=False)

In [7]:
training_data = TerraDataset('/content/drive/MyDrive/Colab Notebooks/TERRa/train.jsonl')
predictions = training_data.predict
train_dataloader = DataLoader(training_data, batch_size = 8, shuffle = True, collate_fn = collator,pin_memory=True)

In [8]:
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [9]:
def cleverslice(predictions,batch_ndx,length):
  if batch_ndx == 0:
    return predictions[0:8]
  if length > ((batch_ndx+1)*8):
    return predictions[batch_ndx*8:(batch_ndx+1)*8]
  else:
    return predictions[batch_ndx*8:length]

In [10]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        predicted = torch.sigmoid(self.linear(x))
        return predicted

In [11]:
length = len(predictions)
#define our loss function Binary cross entropy
loss_fn = torch.nn.BCELoss()
#logreg
modus = LogisticRegression(768,1)
#stohastic gradient descent
optimizer = torch.optim.SGD(modus.parameters(), lr=1e-4)


In [14]:
graph = []
epochs = 1
for epoch in range(epochs):
  for batch_ndx, (sample) in enumerate(train_dataloader):
      a = model(input_ids = sample['input_ids'],attention_mask = sample['attention_mask'],return_dict = True,output_hidden_states = True)['hidden_states'][-1]
      #take last hidden layer for last word (it is res)
      l = sample['length']-1
      l = l.unsqueeze(-1)
      ind = l.repeat(1,768)
      ind = ind.unsqueeze(1)
      res = torch.gather(a, 1, ind)
      #predict classes
      y_predicted = modus(res).squeeze()
      y_predicted = y_predicted.float()
      #find loss function (cleverslice is iterator(batcher) on true predicts) 
      loss = loss_fn(y_predicted,cleverslice(predictions,batch_ndx,length).float())
      loss.backward()
      graph.append(loss.item())
      optimizer.step()
      optimizer.zero_grad()
      #print some information
      if batch_ndx % 10 == 0 or batch_ndx == 0:
        loss, current = loss.item(), batch_ndx
        print(f"loss: {loss}  [{epoch}][{current}]")

loss: 0.7600924968719482  [0][0]
loss: 0.7074878811836243  [0][10]
loss: 0.6566055417060852  [0][20]
loss: 0.6728969812393188  [0][30]
loss: 0.727428674697876  [0][40]
loss: 0.7526039481163025  [0][50]
loss: 0.7203337550163269  [0][60]


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(graph,range(len(graph)))